In [20]:
import pandas as pd
import cvxpy as cp
import numpy as np

euro_calls = [
    (210, 15.21),
    (215, 11.45),
    (220, 7.93),
    (225, 5.1),
    (230, 3.05),
    (235, 1.71),
    (240, 0.91),
    (245, 0.51),
]

euro_puts = [
    (210, 1.53),
    (215, 2.53),
    (220, 3.9),
    (225, 6.6),
    (230, 9.84),
    (235, 14.16),
    (240, 17.73),
]

zero_coupon_bond = 0.997

apple_stock = 222.72

outcomes = np.arange(200, 277.5, 2.5)

all_products = euro_calls + euro_puts + [(0, zero_coupon_bond), (0, apple_stock)]

x = cp.Variable(len(all_products))
p = [prod[1] for prod in all_products]

payoff_matrix = np.zeros((len(outcomes), len(all_products)))
for i, (strike, price) in enumerate(euro_calls):
    payoff_matrix[:, i] = np.maximum(outcomes - strike, 0)
for i, (strike, price) in enumerate(euro_puts):
    payoff_matrix[:, i + len(euro_calls)] = np.maximum(strike - outcomes, 0)
    
payoff_matrix[:, -2] = 1
payoff_matrix[:, -1] = outcomes
    
print(payoff_matrix)

[[  0.    0.    0.    0.    0.    0.    0.    0.   10.   15.   20.   25.
   30.   35.   40.    1.  200. ]
 [  0.    0.    0.    0.    0.    0.    0.    0.    7.5  12.5  17.5  22.5
   27.5  32.5  37.5   1.  202.5]
 [  0.    0.    0.    0.    0.    0.    0.    0.    5.   10.   15.   20.
   25.   30.   35.    1.  205. ]
 [  0.    0.    0.    0.    0.    0.    0.    0.    2.5   7.5  12.5  17.5
   22.5  27.5  32.5   1.  207.5]
 [  0.    0.    0.    0.    0.    0.    0.    0.    0.    5.   10.   15.
   20.   25.   30.    1.  210. ]
 [  2.5   0.    0.    0.    0.    0.    0.    0.    0.    2.5   7.5  12.5
   17.5  22.5  27.5   1.  212.5]
 [  5.    0.    0.    0.    0.    0.    0.    0.    0.    0.    5.   10.
   15.   20.   25.    1.  215. ]
 [  7.5   2.5   0.    0.    0.    0.    0.    0.    0.    0.    2.5   7.5
   12.5  17.5  22.5   1.  217.5]
 [ 10.    5.    0.    0.    0.    0.    0.    0.    0.    0.    0.    5.
   10.   15.   20.    1.  220. ]
 [ 12.5   7.5   2.5   0.    0.    0.    0.

In [21]:
x = cp.Variable(len(all_products))
arb_objective = cp.Minimize(p@x)
arb_constraints = [
    payoff_matrix @ x >= 0,
    p@x == -1,
]
arb_problem = cp.Problem(arb_objective, arb_constraints)
arb_problem.solve()

print(f"Ideal portfolio: {x.value}")
print(f"Portfolio cost: {p@x.value}")
print(f"Portfolio payoffs: {payoff_matrix @ x.value}")
print("Since all the payoffs are non-negative, this is an arbitrage portfolio")

Ideal portfolio: [-1.77741454e+00 -4.18990414e+00 -5.61401441e+00 -6.95655963e-01
  2.01388036e+00  7.91076197e+00  1.84246500e+00 -7.83358484e-06
  1.78413547e+00  4.18707147e+00  5.61451308e+00  6.95566473e-01
 -2.01386488e+00 -7.91076878e+00 -1.84246043e+00  9.87342442e+01
  5.70136587e-01]
Portfolio cost: -0.9999999999999432
Portfolio payoffs: [12.09714053 12.23700102 12.3768615  12.51672198 12.65658247 12.81324527
 12.96990807 13.1194892  13.26907033 13.41989814 13.57072594 13.72133003
 13.87193411 14.02257688 14.17321965 14.32384539 14.47447114 14.6251083
 14.77574546 14.92636304 15.07698061 15.22759819 15.37821576 15.52883334
 15.67945092 15.83006849 15.98068607 16.13130365 16.28192122 16.4325388
 16.58315638]
Since all the payoffs are non-negative, this is an arbitrage portfolio


In [22]:
x = cp.Variable(len(all_products))
transaction_costs = np.zeros(len(all_products))
transaction_costs[-2] = 0.001
transaction_costs[-1] = 0.5
for i in range(len(euro_calls) + len(euro_puts)):
    transaction_costs[i] = 0.5
z = cp.Variable(len(all_products))
arb_objective = cp.Minimize(p@x)
arb_constraints = [
    payoff_matrix @ x >= 0,
    p@x + transaction_costs@z == -1,
    z >= x,
    z >= -x,
]
arb_problem = cp.Problem(arb_objective, arb_constraints)
arb_problem.solve()

print(f"Ideal portfolio: {x.value}")

if x.value is None:
    print(f"There is no arbitrage")
else:
    print(f"Ideal portfolio: {x.value}")
    print(f"Portfolio cost: {p@x.value}")
    print(f"Portfolio payoffs: {payoff_matrix @ x.value}")


Ideal portfolio: None
There is no arbitrage


This example demonstrates that transaction costs can make arbitrage unprofitable.
This is one of multiple ways in which transaction costs can stabilize markets:

Transaction costs can play a stabilizing role in markets by acting as a deterrent to excessive trading and discouraging speculative behavior. When transaction costs are present, they create a natural "friction" in the trading process, making it costly for traders to rapidly buy and sell assets in response to minor price changes. This can have a few key stabilizing effects:

Reduction of Excessive Volatility: Transaction costs make short-term, high-frequency trading less profitable. Without these costs, traders might respond to every small price movement, potentially leading to more volatility as prices react more to short-term fluctuations than to fundamental information.

Encouragement of Long-Term Investment: By discouraging frequent trading, transaction costs promote a longer-term investment approach. When investors are more focused on holding assets over time rather than rapidly shifting positions, the market tends to reflect more fundamental values and long-term trends, rather than short-term noise, reducing the chances of erratic price movements.

Decrease in Herd Behavior: Transaction costs can also help curb herd behavior, where traders collectively follow trends without considering underlying value. In a low-cost environment, it’s easy for traders to jump on trends or "bandwagons," which can amplify market swings. Higher transaction costs can help moderate this behavior by making it less profitable to join in on every trend.


    


In [23]:
x = cp.Variable(len(all_products))
z = cp.Variable(len(all_products))
put_245_payoff = np.maximum(245 - outcomes, 0)

rep_objective = cp.Minimize(p@x + transaction_costs@z)
rep_constraints = [
    payoff_matrix @ x >= put_245_payoff,
    z >= x,
    z >= -x,
]
rep_problem = cp.Problem(rep_objective, rep_constraints)
rep_problem.solve()

print(f"Replicate the payoff of a put option with strike 245:")
print(f"Optimal portfolio: {x.value}")
print(f"Portfolio cost: {np.sum(p@x.value)}")

Replicate the payoff of a put option with strike 245:
Optimal portfolio: [-1.20114843e-08 -2.13592590e-08 -9.99999796e-01 -2.63690045e-09
  3.83962025e-12  2.32552029e-09  9.51390246e-09  9.99999797e-01
  2.55878890e-09  1.71884669e-08  9.99999805e-01  1.78910129e-09
  1.30765988e-09 -3.45886573e-09  1.65808794e-07  2.49999978e+01
 -4.17584229e-09]
Portfolio cost: 21.40500021101272
